In [7]:
!bokeh serve --show app.py


2025-02-23 20:16:53,421 Starting Bokeh server version 3.6.3 (running on Tornado 6.4.2)
2025-02-23 20:16:53,424 User authentication hooks NOT provided (default user enabled)
2025-02-23 20:16:53,427 Bokeh app running at: http://localhost:5006/app
2025-02-23 20:16:53,428 Starting Bokeh server with process id: 12238
2025-02-23 20:16:54,850 WebSocket connection opened
2025-02-23 20:16:54,857 ServerConnection created


In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
!bokeh serve --show app3.py

2025-03-22 14:55:12,788 Starting Bokeh server version 3.6.3 (running on Tornado 6.4.2)
2025-03-22 14:55:12,799 User authentication hooks NOT provided (default user enabled)
2025-03-22 14:55:12,812 Bokeh app running at: http://localhost:5006/app3
2025-03-22 14:55:12,813 Starting Bokeh server with process id: 91199
[[0.4036406  0.44083156 0.42433878 0.41219409 0.29906975 0.35523799]
 [0.33026653 0.473891   0.40775897 0.38460824 0.32313851 0.34056775]
 [0.39434632 0.48117773 0.36945959 0.40694596 0.32819784 0.32244838]
 [0.35998744 0.43158295 0.35672841 0.38299284 0.2864201  0.29865834]
 [0.30628874 0.40331083 0.35696314 0.38754915 0.29975411 0.32440495]]
2025-03-22 14:55:24,231 W-1000 (MISSING_RENDERERS): Plot has no renderers: figure(id='p1092', ...)
2025-03-22 14:55:24,231 W-1000 (MISSING_RENDERERS): Plot has no renderers: figure(id='p1162', ...)
2025-03-22 14:55:24,916 WebSocket connection opened
2025-03-22 14:55:24,917 ServerConnection created
^C

Interrupted, shutting down


In [34]:
 np.random.random((10, 10))

array([[0.08786845, 0.54132165, 0.59286759, 0.69970678, 0.03287814,
        0.79286123, 0.74491595, 0.76006483, 0.93879585, 0.664171  ],
       [0.11353637, 0.70644525, 0.48613955, 0.95604554, 0.50786784,
        0.84685765, 0.98462476, 0.89144011, 0.45740793, 0.86925753],
       [0.82520584, 0.19425177, 0.10770322, 0.20743559, 0.87667629,
        0.13240189, 0.15618961, 0.28875101, 0.9437995 , 0.76899678],
       [0.36702704, 0.01133164, 0.42159591, 0.14231964, 0.57744893,
        0.88174641, 0.74963369, 0.41413907, 0.86131414, 0.56396494],
       [0.81213393, 0.41271461, 0.68326212, 0.05049474, 0.82685966,
        0.53653533, 0.4156695 , 0.54013372, 0.95555806, 0.32664203],
       [0.53300127, 0.56068645, 0.93901128, 0.17493331, 0.48067407,
        0.8409843 , 0.18180876, 0.54753552, 0.46537592, 0.05697585],
       [0.49031141, 0.423084  , 0.75863195, 0.34824488, 0.42110368,
        0.69551042, 0.74667563, 0.20162895, 0.34015811, 0.22343285],
       [0.50284095, 0.21163822, 0.8533258

In [3]:
import pickle
with open("data/fid_model.pkl", 'rb') as f:
    fid_model = pickle.load(f)

In [28]:
import numpy as np
np.random.random((10, 10)).shape

(10, 10)

In [30]:
from sharp_ratio import sharp_grid
np.shape(sharp_grid(df_returns_fake))

(5, 6)

In [2]:
import pandas as pd

df_returns_real=pd.read_csv("data/real_data.csv", index_col=0)
df_returns_fake=pd.read_csv("data/fake_data.csv", index_col=0)

In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
from fid import calculate_fid

calculate_fid(df_returns_real, df_returns_fake)

с


0.008528099869022938

In [5]:
df_returns_real.shape

(2706, 5)

In [6]:
df_returns_fake.shape

(2706, 5)

In [7]:
import torch

In [8]:
from statsmodels.tsa.stattools import acf
from scipy.signal import argrelextrema
import numpy as np
import pandas as pd
import pickle
from scipy.linalg import sqrtm
from TSGBench.src.ts2vec import initialize_ts2vec
import torch

def find_length(data):
    if len(data.shape)>1:
        return 0
    data = data[:min(20000, len(data))]
    base = 3
    nobs = len(data)
    nlags = int(min(10 * np.log10(nobs), nobs - 1))
    auto_corr = acf(data, nlags=nlags, fft=True)[base:]
    local_max = argrelextrema(auto_corr, np.greater)[0]
    try:
        max_local_max = np.argmax([auto_corr[lcm] for lcm in local_max])
        if local_max[max_local_max]<3 or local_max[max_local_max]>300:
            return 125
        return local_max[max_local_max]+base
    except Exception as e:
        print(e)
        return 125

def preprocess(data, step=1):
    window_all = []
    for i in range(data.shape[1]):
        window_all.append(find_length(data[:,i]))

    seq_length = int(np.mean(np.array(window_all)))

    window_size = seq_length
    
    if data.ndim != 2:
        raise ValueError("Input array must be 2D")
    L, C = data.shape  # Length and Channels
    if L < window_size:
        raise ValueError("Window size must be less than or equal to the length of the array")

    # Calculate the number of windows B
    B = L - window_size + 1
    
    # Shape of the output array
    new_shape = (B, window_size, C)
    
    # Calculate strides
    original_strides = data.strides
    new_strides = (original_strides[0],) + original_strides  # (stride for L, stride for W, stride for C)

    # Create the sliding window view
    strided_array = np.lib.stride_tricks.as_strided(data, shape=new_shape, strides=new_strides)
    return strided_array


    
def calculate_fid(ori_data, gen_data):
    print("с")
    ori_data = ori_data.to_numpy()
    gen_data = gen_data.to_numpy()
    ori_data = preprocess(ori_data)
    gen_data = preprocess(gen_data)
    fid_model = initialize_ts2vec(np.transpose(ori_data, (0, 2, 1)),torch.device('cpu'))
    ori_repr = fid_model.encode(np.transpose(ori_data,(0, 2, 1)), encoding_window='full_series')
    gen_repr = fid_model.encode(np.transpose(gen_data,(0, 2, 1)), encoding_window='full_series')
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

def evaluate_data(ori_data, gen_data):
    train_data = ori_data
    fid_model = initialize_ts2vec(np.transpose(train_data, (0, 2, 1)),torch.device('cpu'))
    ori_repr = fid_model.encode(np.transpose(ori_data,(0, 2, 1)), encoding_window='full_series')
    gen_repr = fid_model.encode(np.transpose(gen_data,(0, 2, 1)), encoding_window='full_series')
    cfid = calculate_fid(ori_repr,gen_repr)
    return cfid

In [9]:
ori_data = df_returns_real.to_numpy()
gen_data = df_returns_fake.to_numpy()
ori_data = preprocess(ori_data)
gen_data = preprocess(gen_data)
fid_model = initialize_ts2vec(np.transpose(ori_data, (0, 2, 1)),torch.device('cpu'))
ori_repr = fid_model.encode(np.transpose(ori_data,(0, 2, 1)), encoding_window='full_series')
gen_repr = fid_model.encode(np.transpose(gen_data,(0, 2, 1)), encoding_window='full_series')
# calculate mean and covariance statistics
mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
# calculate sum squared difference between means
ssdiff = np.sum((mu1 - mu2)**2.0)
# calculate sqrt of product between cov
covmean = sqrtm(sigma1.dot(sigma2))
# check and correct imaginary numbers from sqrt
if np.iscomplexobj(covmean):
    covmean = covmean.real
# calculate score
fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (40x12 and 43x64)

In [10]:
gen_data.shape

(2695, 12, 5)

In [11]:
ori_data.shape

(2664, 43, 5)